 <img src="Events.png" width="320"> 
 
# Events

The whole canvas and named components can respond to mouse and
keyboard events.  Canvases and reference frames provide methods
for converting event pixel offsets into x and y coordinates
relative to the frame of reference.  Events over a named object
provide the properties of the named object as `event.object_info`.

In the demo below a global event handler tracks the mouse position
and individual elements drawn on the canvas respond to mouse
click events using local event handlers.

In [1]:
from jp_doodle import dual_canvas
from IPython.display import display

In [2]:
# In this demonstration we do most of the work in Javascript.

demo = dual_canvas.DualCanvasWidget(width=320, height=220)
display(demo)

demo.js_init("""

// A floating tooltip (not a canvas object)
var tooltip = $("<div>tooltip here</div>").appendTo(element);
tooltip.css({
    position: "absolute",
    width: "140px",
    height: "auto",
    background: "#ddd",
    font: "12px sans-serif",
    opacity: 0
});

// Draw some named elements on the canvas.
// A filled yellow circle (disk) named "Colonel Mustard
element.circle({name: "Colonel Mustard", x:100, y:150, r:90, color:"yellow"});

// A filled red rectangle named "Miss Scarlett"
element.rect({name: "Miss Scarlett", x:100, y:130, w:100, h:20, color: "red"});

// An unfilled white circle named "Mrs. White"
element.circle({
    name: "Mrs. White", x:100, y:150, r:58, fill:false, 
    color:"white", lineWidth: 14});
    
// An unfilled blue rectangle named Mrs. Peacock
element.rect({
    name: "Mrs. Peacock", x:40, y:110, w:100, h:20,
    color: "blue", lineWidth: 10, degrees:70, fill:false});

// A line segment named "Professor Plum".
element.line({
    name: "Professor Plum", x1:190, y1:100, x2:10, y2:200,
    color:"purple", lineWidth: 20});
    
// A click indicator circle, initially hidden
element.circle({
    name: "click indicator", x:0, y:0, r:3, hide:true, color:"#999"
});
    
var say_on_click = function(phrase) {
    // generate a local callback handler
    return function (event) {
        var object_name = event.object_info.name;
        var element_offset = element.visible_canvas.offset();
        var canvas_location = element.event_model_location(event);
        var pixel_offset = element.event_pixel_location(event);
        var name = event.canvas_name;
        element.change("click indicator", {hide:false});
        element.transition("click indicator", 
            {x: canvas_location.x, y: canvas_location.y, r:3, color:"#999"});
        tooltip.offset({
            left: pixel_offset.x + element_offset.left + 10,
            top: pixel_offset.y + element_offset.top + 10,
        })
        tooltip.html("<b>"+object_name + "</b> says <br/><em>" + phrase + "</em>");
        tooltip.css({opacity: 0.8});
    };
};
// Attach some click handlers to the objects.
element.on_canvas_event("click", say_on_click("By Jove!"), "Colonel Mustard");
element.on_canvas_event("click", say_on_click("My Word!"), "Miss Scarlett");
element.on_canvas_event("click", say_on_click("Goodness!"), "Mrs. Peacock");
element.on_canvas_event("click", say_on_click("Gad!"), "Professor Plum");
element.on_canvas_event("click", say_on_click("Saints preserve us!"), "Mrs. White");

// Background click makes the dialog and click indicator disappear
element.on_canvas_event("click", function() {
    tooltip.css({opacity: 0});
    element.transition("click indicator", {r:50, color:"rgba(255,255,255,1)", x:-100});
    });

// Report position on mouse move
var info_div = $("<div>Click an object for a response.</div>").appendTo(element);

element.on_canvas_event("mousemove", function(event) {
    var canvas_location = element.event_model_location(event);
    info_div.html("<div>mouse move at " + Math.round(canvas_location.x) +", "+Math.round(canvas_location.y)+"</div>")
})

// Fit the figure into the available space
element.fit(null, 10);

$("<button>Silence Col. Mustard</button>")
.appendTo(element)
.click(function() {
    element.off_canvas_event("click", "Colonel Mustard");
});

var saved_events = null;  // keep events for re-enabling later.

$("<button>Disable events</button>")
.appendTo(element)
.click(function() {
    saved_events = element.reset_events();
});

$("<button>Re-enable events</button>")
.appendTo(element)
.click(function() {
    if (saved_events) {
        element.restore_events(saved_events);
    }
});
""")

DualCanvasWidget(status=u'Not yet rendered')

# Turning events off

As demonstrated above an event may be turned off using the `off_canvas_event` method.

All events can be temporarily disabled using the `reset_events` methods which returns
the internal event archive structure and replaces the internal structure
with a new empty structure as shown above.  After events have been disabled
new handlers may be added to the canvas without altering the saved event structure.
The events can be restored using the
`restore_events(saved_events)` method as shown above
which will have the side effect of cancelling any events added since the reset.  

Disabling and restoring events
can be used to switch a canvas between different modes -- for example to switch between
a "normal" and an "edit" mode.


# Events calling back to Python in Jupyter notebooks

In the Jupyter notebook context a canvas widget event may directly or indirectly
trigger a call-back to the Python kernel.  The Python kernel may modify the canvas in
response to the event.

In the example below we modify the demo shown above to look up values and compute the
time in the Python interpreter in response to mouse click events.

In [3]:
import time

demo2 = dual_canvas.DualCanvasWidget(width=320, height=220)
display(demo2)

exclamations = {
    "Colonel Mustard": "Who goes there?",
    "Miss Scarlett": "You are standing on my foot!",
    "Mrs. Peacock": "The indignity!",
    "Professor Plum": "En Garde!",
    "Mrs. White": "Faith and Begorrah!",
}

def python_call_back(name):
    "Python call back triggered by click event."
    if not name:
        name = "Nobody"
    text = (
        "<b>" + repr(name) + "</b>" + " says " + 
        "<em>" + repr(exclamations.get(name, "???")) + "</em>"
        "<br/> at " + 
        time.ctime()
    )
    # call in to javascript to display the string
    demo2.element.javascript_call_in(text)
    
# Attach (a proxy to) the Python callback to the widget element
# as element.python_call_back
#
demo2.set_element("python_call_back", python_call_back)

demo2.js_init("""

// A floating tooltip (not a canvas object).
var tooltip = $("<div>tooltip here</div>").appendTo(element);
tooltip.css({
    position: "absolute",
    width: "140px",
    height: "auto",
    background: "#ddd",
    font: "12px sans-serif",
    opacity: 0
});

// Javascript call_in called by Python call back.
element.javascript_call_in = function(text) {
    tooltip.html("<div>" + text + "</div>");
};

// Define a click handler which calls back to Python.
var click_handler = function (event) {
    var element_offset = element.visible_canvas.offset();
    var canvas_location = element.event_model_location(event);
    var pixel_offset = element.event_pixel_location(event);
    var name = event.canvas_name || null;
    element.change("click indicator", {hide:false});
    element.transition("click indicator", 
        {x: canvas_location.x, y: canvas_location.y, r:3, color:"#999"});
    tooltip.offset({
        left: pixel_offset.x + element_offset.left + 10,
        top: pixel_offset.y + element_offset.top + 10,
    })
    tooltip.css({opacity: 0.8});
    
    // Call the python callback to compute the time and look up the exclamation.
    element.python_call_back(name);
};

// Attach the click handler to the whole canvas.
element.on_canvas_event("click", click_handler);

// Draw some named elements on the canvas.
// A filled yellow circle (disk) named "Colonel Mustard
element.circle({name: "Colonel Mustard", x:100, y:150, r:90, color:"yellow"});

// A filled red rectangle named "Miss Scarlett"
element.rect({name: "Miss Scarlett", x:100, y:130, w:100, h:20, color: "red"});

// An unfilled white circle named "Mrs. White"
element.circle({
    name: "Mrs. White", x:100, y:150, r:58, fill:false, 
    color:"white", lineWidth: 14});
    
// An unfilled blue rectangle named Mrs. Peacock
element.rect({
    name: "Mrs. Peacock", x:40, y:110, w:100, h:20,
    color: "blue", lineWidth: 10, degrees:70, fill:false});

// A line segment named "Professor Plum".
element.line({
    name: "Professor Plum", x1:190, y1:100, x2:10, y2:200,
    color:"purple", lineWidth: 20});
    
// A click indicator circle, initially hidden
element.circle({
    name: "click indicator", x:0, y:0, r:3, hide:true, color:"#999"
});

// Fit the figure into the available space
element.fit(null, 10);
""")

DualCanvasWidget(status=u'Not yet rendered')